In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-10"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10314,2024-04-10,Brasil Nbb,19:30,União Corinthians,Pato,2.06,1.69,160.5,1.81,1.85,1.5,1.91,1.56,4jtsxFF5,0.485437,0.591716,0.552486,0.540541,0.077153,0.8,0.6,NaN,NaN,222.014,93.960520,0.423219,240.000,173.930404,0.724710,160.152,317.160,180.20,536.25,0.0,1.0,0.0,0.0,0.139536,0.015456,0.142644,0.86,0.172,6.162791,0.450543,0.3,-0.150543,-1.35,-0.270,-2.555556,0.331935,0.3,-0.031935
10315,2024-04-10,Brasil Nbb,20:00,Botafogo,Pinheiros,2.18,1.64,163.5,1.87,1.79,1.5,2.02,1.50,nFVoyZUB,0.458716,0.609756,0.534759,0.558659,0.068472,0.4,0.0,NaN,NaN,351.900,131.615741,0.374015,167.258,59.532503,0.355932,250.768,153.626,550.68,168.81,1.0,0.0,0.0,0.0,0.199915,0.030912,0.208918,-5.00,-1.000,-1.180000,0.400262,0.3,-0.100262,1.72,0.344,1.860465,0.414118,0.4,-0.014118
10316,2024-04-10,Dinamarca Kvindebasketligaen Feminina,14:00,Aabyhoj F,BMS Herlev F,1.69,2.06,130.5,1.85,1.85,-1.5,1.80,2.27,rBpU1afn,0.591716,0.485437,0.540541,0.540541,0.077153,0.0,0.0,NaN,NaN,NaN,NaN,NaN,191.356,119.216377,0.623008,0.000,0.000,315.70,378.87,0.0,0.0,0.0,0.0,0.139536,0.000000,0.163312,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10317,2024-04-10,Espanha Leb Ouro,15:00,Real Bétis,Lleida,1.65,2.15,159.5,1.85,1.85,-2.5,1.81,2.32,Eg9vmG0N,0.606061,0.465116,0.540541,0.540541,0.071177,0.6,0.2,NaN,NaN,183.368,97.295923,0.530605,144.954,16.707242,0.115259,115.428,137.438,85.68,123.75,0.0,0.0,0.0,0.0,0.186081,0.000000,0.174637,0.30,0.060,10.833333,0.000000,0.0,0.000000,3.71,0.742,1.549865,0.000000,0.0,0.000000
10318,2024-04-10,Espanha Leb Ouro,15:15,Melilla,Hestia Menorca,1.75,1.98,152.5,1.87,1.79,-2.5,2.02,2.38,IL4AsfOp,0.571429,0.505051,0.534759,0.558659,0.076479,0.4,0.8,NaN,NaN,250.616,161.737828,0.645361,241.804,98.439475,0.407104,178.312,277.318,158.76,398.13,0.0,0.0,0.0,0.0,0.087204,0.030912,0.115708,0.92,0.184,4.076087,0.000000,0.0,0.000000,4.50,0.900,1.088889,0.000000,0.0,0.000000
10319,2024-04-10,Espanha Leb Ouro,15:30,Clavijo,Estela,1.75,1.98,151.5,1.80,1.86,-2.5,2.02,2.38,z3AzldoH,0.571429,0.505051,0.555556,0.537634,0.076479,0.0,0.4,NaN,NaN,406.832,136.253919,0.334914,165.238,56.608458,0.342587,219.286,276.578,311.86,135.05,0.0,0.0,0.0,0.0,0.087204,0.023184,0.115708,-5.00,-1.000,-0.750000,0.000000,0.0,0.000000,-3.15,-0.630,-1.555556,0.000000,0.0,0.000000
10320,2024-04-10,Europa Liga Bnxt,14:45,Feyenoord,Kortrijk,2.02,1.72,157.5,1.82,1.88,1.5,1.88,1.63,fmD0JAy3,0.495050,0.581395,0.549451,0.531915,0.076445,0.6,0.6,NaN,NaN,322.574,324.189124,1.005007,234.688,142.887968,0.608842,142.160,258.544,138.88,404.68,0.0,0.0,0.0,0.0,0.113440,0.022933,0.100727,-3.39,-0.678,-1.504425,0.000000,0.0,0.000000,-2.66,-0.532,-1.353383,0.000000,0.0,0.000000
10321,2024-04-10,Filipinas Copa Das Filipinas,08:30,Magnolia Hotshots,NorthPort,1.55,2.39,183.5,1.86,1.88,-4.5,1.91,3.00,00l49H3b,0.645161,0.418410,0.537634,0.531915,0.063571,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.301507,0.007563,0.313950,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10322,2024-04-10,Israel Liga Leumit,12:50,Safed,Elitzur Shomron,1.69,2.06,163.5,1.83,1.87,-2.5,1.93,2.24,00ACo4hC,0.591716,0.485437,0.546448,0.534759,0.077153,0.6,0.0,NaN,NaN,151.284,21.241079,0.140405,201.718,74.508091,0.369368,157.426,0.000,163.18,297.90,0.0,0.0,0.0,0.0,0.139536,0.015289,0.105133,0.54,0.108,6.388889,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1032

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Dinamarca Kvindebasketligaen Feminina,Aabyhoj F,BMS Herlev F,130.5,1.85,1.0000,Over
1,15:00,Espanha Leb Ouro,Real Bétis,Lleida,159.5,1.85,1.0000,Over
2,15:15,Espanha Leb Ouro,Melilla,Hestia Menorca,152.5,1.87,1.0000,Over
3,15:30,Espanha Leb Ouro,Clavijo,Estela,151.5,1.80,1.0000,Over
4,08:30,Filipinas Copa Das Filipinas,Magnolia Hotshots,NorthPort,183.5,1.86,1.0000,Over
5,12:50,Israel Liga Leumit,Safed,Elitzur Shomron,163.5,1.83,1.0000,Over
6,07:05,Japão B.League,Brave Thunders,Sun Rockers Shibuya,153.5,1.85,1.0000,Over
7,07:35,Japão B.League,Ryukyu,Diamond Dolphins,163.5,1.86,1.0000,Over
8,12:00,Rússia Liga Vtb United,Saratov,Uralmash Ekaterinburg,159.5,1.85,1.0000,Over
9,13:00,Rússia Liga Vtb United,Nizhny Novgorod,Samara,152.5,1.81,1.0000,Over
